In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection

from recsys import preprocess
from recsys import utility

RANDOM_STATE = 2342

%matplotlib inline

In [2]:
train = pd.read_csv('data/train_final.csv', delimiter='\t')
playlists = pd.read_csv('data/playlists_final.csv', delimiter='\t')
target_playlists = pd.read_csv('data/target_playlists.csv', delimiter='\t')
target_tracks = pd.read_csv('data/target_tracks.csv', delimiter = '\t')
tracks = pd.read_csv('data/tracks_final.csv', delimiter='\t')

In [3]:
train, test = model_selection.train_test_split(train, test_size=0.20, random_state=RANDOM_STATE)

In [8]:
small_train = train.loc[1:1000,:]

In [12]:
s = small_train.groupby('track_id')

In [22]:
a = s.count()

In [27]:
a.sort_values('playlist_id', ascending=False, inplace=True)

In [234]:
df.to_csv?

In [39]:
a = pd.DataFrame(columns=['a', 'b'])

In [ ]:
train = get_train()
target_playlist = get_target_playlists()
target_tracks = get_target_tracks()

train, test = model_selection.train_test_split(train, test_size=0.20, random_state=RANDOM_STATE)

In [29]:
most_popular = get_most_popular_tracks(train)
tracks_in_playlist = get_playlist_track_list(train)

In [32]:
most_popular = most_popular.index.values

In [41]:
from recsys.preprocess import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection

RANDOM_STATE = 42

tracks_to_suggest = np.intersect1d(most_popular, target_tracks)
predictions = []

for it,row in target_playlist.iterrows():
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist[row['playlist_id']]:
            # Predict track i
            pred.append(most_popular[i])
            count += 1
        i += 1
    predictions.append(' '.join(map(str, pred)))

In [43]:
results = pd.DataFrame(np.transpose([target_playlist['playlist_id'].values, predictions]), columns=['playlist_id', 'track_ids'])

In [46]:
len(test)

208105

In [47]:
from recsys.utility import *

In [48]:
def evaluate(test, recommendations):
    test_good = preprocess.get_playlist_track_list(test)

    mean_ap = 0
    for pl_id, tracks in recommendations:
        correct = 0
        ap = 0
        for it, t in enumerate(tracks):
            if t in test_good[pl_id]:
                correct += 1
                ap += correct / (it+1)
        ap /= len(tracks)
        mean_ap += ap

    return mean_ap / len(recommendations)


In [80]:
recomm = np.array([list(map(int,x.split(' '))) for x in results['track_ids'].values])

In [81]:
recomm

array([[1563309, 1363985, 3705881, 1595978, 3779477],
       [1563309, 1363985, 3705881, 1595978, 3779477],
       [1563309, 1363985, 3705881, 1595978, 3779477],
       ..., 
       [1563309, 1363985, 3705881, 1595978, 3779477],
       [1563309, 1363985, 3705881, 1595978, 3779477],
       [1563309, 1363985, 3705881, 1595978, 3779477]])

In [82]:
evaluate(test, recomm)

ValueError: too many values to unpack (expected 2)

In [225]:
def train_test_split(train, test_size=0.3):
    playlists = train.groupby('playlist_id').count()
    to_choose_playlists = playlists[playlists['track_id'] > 7].index.values
    target_playlists = np.random.choice(to_choose_playlists, replace=False, size=int(test_size * len(to_choose_playlists)))
    
    target_tracks = np.array([])
    indexes = np.array([])
    for p in target_playlists:
        selected_df = train[train['playlist_id'] == p].sample(5)
        selected_tracks = selected_df['track_id'].values
        target_tracks = np.union1d(target_tracks, selected_tracks)
        indexes = np.union1d(indexes, selected_df.index.values)
    
    test = df.loc[indexes].copy()
    train = train.drop(indexes)
    
    return train, test, target_playlists, target_tracks
    

In [226]:
x,y,z,w = train_test_split(train)

In [227]:
len(train)

832417

In [228]:
len(x)

798407

In [229]:
len(y)

34010

In [230]:
len(z)

6802

In [231]:
len(w)

24281

In [233]:
len(y)/len(x)

0.04259732191726776

In [235]:
df.to_csv?

In [237]:
train[train['track_id'] == 252]

,playlist_id,track_id
269276,10550126,252
115607,8283678,252
999950,4322356,252
1007372,5554934,252
477645,1404694,252
405463,6102596,252
78094,7996678,252
701585,3853994,252
268901,11355891,252
590443,7333844,252


In [238]:
most_popular = get_most_popular_tracks(train)

In [241]:
most_popular

,playlist_id
track_id,
1563309,383
1363985,351
3705881,322
1595978,322
3779477,317
204966,311
2863395,301
3166665,292
1580480,290


In [243]:
target_tracks['track_id'].values

array([1316175, 3885714, 3091270, ..., 2739213, 2228646, 2265463])

In [251]:
len(np.intersect1d(most_popular.index.values[:10], target_tracks['track_id'].values))

10

In [247]:
len(target_tracks['track_id'])

32195

In [249]:
len(most_popular.index)

99789

In [252]:
most_popular

,playlist_id
track_id,
1563309,383
1363985,351
3705881,322
1595978,322
3779477,317
204966,311
2863395,301
3166665,292
1580480,290


In [215]:
playlists = train.groupby('playlist_id').count()
to_choose_playlists = playlists[playlists['track_id'] > 7].index.values

In [216]:
len(to_choose_playlists)

22676

In [217]:
len(playlists)

44884

In [185]:
playlists = a.groupby('playlist_id').count()

,track_id
playlist_id,
10732,1
18357,1
18766,1
21452,1
21567,1
22269,1
35761,1
35966,1
41685,1


In [173]:
df.loc[[518209,598021]]

,playlist_id,track_id
518209,4546088,2285993
598021,4685686,3133472


In [131]:
a = np.array([1, 2, 3, 4, 5])

In [139]:
np.union1d(a,b)

array([      3,       4,       5,       6,       7,       8,  252429,
       1952325])

In [152]:
a = df.loc[[518209, 598021]]

In [157]:
len(df.drop([518209]))

832416

In [158]:
len(df)

832417

In [128]:
train[train['playlist_id'] == 6226465].sample(5)['track_id'].values

array([3667257, 2765634, 3001336, 3477637, 1277686])

In [117]:
len(to_choose_playlists)

18337

In [92]:
a = (train.groupby('playlist_id').count())

In [101]:
a[a['track_id'] > 10].index.values

array([    7692,     7816,     7912, ..., 11764165, 11764751, 11764851])

In [38]:
len(target_tracks)

32195

In [39]:
len(most_popular)

99789

In [40]:
len(np.intersect1d(target_tracks, most_popular))

32074